In [1]:
import simpletransformers
from simpletransformers.classification import ClassificationModel, ClassificationArgs

import pandas as pd
import torch
import logging
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
import logging 
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

X_train = pd.read_csv("./data/training/wo_Meta/preprocessed/X_train.csv")
y_train = pd.read_csv("./data/training/wo_Meta/preprocessed/y_train.csv")

X_test = pd.read_csv("./data/training/wo_Meta/preprocessed/X_test.csv")
y_test = pd.read_csv("./data/training/wo_Meta/preprocessed/y_test.csv")

X_valid = pd.read_csv("./data/training/wo_Meta/preprocessed/X_valid.csv")
y_valid = pd.read_csv("./data/training/wo_Meta/preprocessed/y_valid.csv")

X_train_add = pd.read_csv("./data/training/wo_Meta/preprocessed/additional/X_train.csv")
y_train_add = pd.read_csv("./data/training/wo_Meta/preprocessed/additional/y_train.csv")

X_test_add = pd.read_csv("./data/training/wo_Meta/preprocessed/additional/X_test.csv")
y_test_add = pd.read_csv("./data/training/wo_Meta/preprocessed/additional/y_test.csv")


df_train = X_train
df_train["label"] = y_train
df_test = X_test
df_test["label"] = y_test
df_valid = X_valid
df_valid["label"] = y_valid

df_train_add = X_train_add
df_train_add["label"] = y_train_add
df_test_add = X_test_add
df_test_add["label"] = y_test_add


df_valid.drop(columns=["DOI"], inplace=True)
df_train.drop(columns=["DOI"], inplace=True)

In [ ]:
import wandb
wandb.login()

sweep_config = {
    'method': 'random', #grid, random, bayes
    'metric': {
      'name': 'eval_loss',
      'goal': 'minimize'   
    },
    'parameters': {
        'learning_rate': {
            'values': [ 5e-5, 4e-5, 3e-5]
        },
        #'epochs':{
             #'values':[3,4,5,6,7,8,9,10,11,12,13,14,15,20,25,30]
        #},
        'train_batch_size': {
            'values':[8,16,32,64]
        },
        'eval_batch_size': {
            'values':[4,8,16]
        }
    }
}

sweep_id = wandb.sweep(sweep_config,project="nlp-0121")

Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable
wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter:  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: 6fq9dgg5
Sweep URL: https://wandb.ai/eber/nlp-0121/sweeps/6fq9dgg5


In [ ]:
def train():
    wandb.init()
    
    # Training arguments
    train_args = {
        'num_train_epochs' : 30,
        'evaluate_during_training': True,
        'save_eval_checkpoints': False,
        'overwrite_output_dir': True,
        'use_multiprocessing' : True,
        'reprocess_input_data' : True,
        'fp16': False,
        'use_early_stopping' : True,
        'early_stopping_delta' : 0.1,
        'early_stopping_metric' : "mcc",
        'early_stopping_metric_minimize' : False,
        'early_stopping_patience' : 5,
        'train_batch_size': wandb.config.train_batch_size,
        'eval_batch_size': wandb.config.eval_batch_size,
        'learning_rate': wandb.config.learning_rate,
        'wandb_project': "nlp-0121"
    }
    
    model = simpletransformers.classification.ClassificationModel("bert", "microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext", args=train_args, use_cuda=True)

    # Train the model
    model.train_model(df_train, eval_df = df_valid)
    #wandb.save('model.h5')
    PATH = wandb.run._run_id + '.h5'
    torch.save(model, PATH)
    
    # Evaluate the model
    model.eval_model(df_test)
    #wandb.save('model.h5')

wandb.agent(sweep_id,function=train)

wandb: Agent Starting Run: w1d16dg9 with config:
wandb: 	eval_batch_size: 8
wandb: 	learning_rate: 4e-05
wandb: 	train_batch_size: 16
Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable
wandb: Currently logged in as: eber (use `wandb login --relogin` to force relogin)


Some weights of the model checkpoint at monologg/biobert_v1.1_pubmed were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

/opt/conda/lib/python3.6/site-packages/simpletransformers/classification/classification_model.py:395: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


/opt/conda/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)
/opt/conda/lib/python3.6/site-packages/simpletransformers/classification/classification_model.py:975: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."
